In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')
train_labels = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv')

In [3]:
train_labels.head()

,sequence,state
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [4]:
train.head(5)

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.0,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.0,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.0,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.0,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.0,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359


In [5]:
def df_flatten(dataframe):
    dataframe_drop = dataframe.drop(['sequence', 'subject', 'step'], axis = 1)
    
    df = dataframe_drop[0:60]
    lst = [ dataframe.subject[0]]
    
    for i in df.columns:
        temp_lst = df[i].to_list()
        lst += temp_lst
        
    main_df = [lst]
    
    for idx in tqdm(range(60, len(dataframe), 60)): # len(dataframe)
        index_start = idx
        index_end = idx+60
        
        df = dataframe_drop[index_start:index_end]
        lst = [dataframe.subject[idx]]
        
        for i in df.columns:
            temp = df[i].to_list()
            lst += temp
            
        main_df += [lst]
    
    return main_df
    
train_df = df_flatten(train)
test_df = df_flatten(test)

100%|██████████| 12217/12217 [00:08<00:00, 1368.37it/s]


In [6]:
train_X = pd.DataFrame(train_df)
test_X = pd.DataFrame(test_df)

train_y = train_labels.state

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Normalizer

In [8]:
scaler = Normalizer()

In [9]:
train_X_scaled = train_X.drop([0], axis = 1)
test_X_scaled = test_X.drop([0], axis = 1)

In [10]:
train_X_scaled = scaler.fit_transform(train_X_scaled)
test_X_scaled = scaler.transform(test_X_scaled)

In [11]:
train_X_ = pd.concat([train_X[0], pd.DataFrame(train_X_scaled)], axis = 1)
test_X_ = pd.concat([test_X[0], pd.DataFrame(test_X_scaled)], axis = 1)

In [12]:
model = GradientBoostingClassifier(verbose = True)

In [13]:
model.fit(train_X_, train_y)

      Iter       Train Loss   Remaining Time 
         1           1.3786           12.05m
         2           1.3717           11.91m
         3           1.3656           11.77m
         4           1.3604           11.65m
         5           1.3559           11.52m
         6           1.3518           11.43m
         7           1.3481           11.32m
         8           1.3428           11.18m
         9           1.3394           11.08m
        10           1.3361           10.95m
        20           1.3085            9.74m
        30           1.2882            8.53m
        40           1.2713            7.32m
        50           1.2573            6.13m
        60           1.2438            4.91m
        70           1.2321            3.69m
        80           1.2210            2.46m
        90           1.2114            1.23m
       100           1.1999            0.00s


GradientBoostingClassifier(verbose=True)

In [14]:
pred = model.predict(test_X_)

In [15]:
test_X.iloc[:,0]

0        684
1        935
2        924
3        769
4        764
        ... 
12213    980
12214    915
12215    872
12216    835
12217    773
Name: 0, Length: 12218, dtype: int64

In [16]:
submission = sample_submission
submission.state = pred

In [17]:
submission

,sequence,state
0,25968,1
1,25969,0
2,25970,0
3,25971,0
4,25972,1
...,...,...
12213,38181,0
12214,38182,1
12215,38183,0
12216,38184,0


In [18]:
submission.to_csv('submission.csv', index = False)